In [1]:
!pip install pygame

In [1]:
import pygame
import random
import math

pygame 2.1.3 (SDL 2.0.22, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Set up the display
WINDOW_WIDTH = 600 # Square dimension
WINDOW = pygame.display.set_mode((WINDOW_WIDTH,WINDOW_WIDTH))
pygame.display.set_caption("Maze solver")

# Colors for path finder visualization
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PURPLE = (128, 0, 128)
ORANGE= (255, 165, 0)
GREY = (128, 128, 128)

In [3]:
def norm2(node1,node2):
  row1,col1 = node1
  row2,col2 = node2
  # Computes pitagorean distance among two nodes
  return (((row2-row1)**2+(col2-col1)**2)**0.5)

In [4]:
def get_clicked_pos(pos, rows):
        # Returns the row and column in the pygame window corresponing to the clicked position

        gap = WINDOW_WIDTH//rows
        x,y = pos
        row = y//gap
        col = x//gap
        return (row,col)

In [5]:
class Node():
    # White nodes are free
    # Black nodes are obstacles
    # Blue node is start
    # Orange node is goal
    # Red nodes are valid nodes
    # Purple nodes are part of final path

    def __init__(self, row, col, radius):
        self.row = row
        self.col = col
        # Variables to draw the circles inside the display
        self.radius = radius//2
        self.x = col*radius
        self.y = row*radius
        # Node color not defined
        self.color = None
    
    def get_pos(self):
      # Outputs coordinates as a tuple
      return (self.row,self.col)

    def is_free(self):
        return self.color == WHITE
    
    def is_obstacle(self):
        return self.color == BLACK

    def is_start(self):
      return self.color == BLUE

    def is_goal(self):
        return self.color == ORANGE

    def set_free(self):
        self.color = WHITE
    
    def set_obstacle(self):
        self.color = BLACK

    def set_node(self):
        self.color = RED

    def set_start(self):
        self.color = BLUE

    def set_goal(self):
        self.color = ORANGE

    def set_path(self):
        self.color = PURPLE
    
    def draw_node(self):
      # Draws the node in the display
      pygame.draw.circle(WINDOW, self.color, (self.x,self.y), self.radius)
      pygame.display.update()

In [6]:
class Maze():
  def __init__(self, ROWS, COLUMNS,N_OBSTACLES,step_size):
    self.map = [] # Initialize map
    # Store map dimensions
    self.rows = ROWS 
    self.cols = COLUMNS
    # Number of obstacles to be initialized
    self.obstacles_number = N_OBSTACLES
    # No start and end node set yet
    self.start = None
    self.end = None
    # Max distance of the new node from its nearest node
    self.step_size = step_size
    self.goal_found = False
    # Dictionary for RRT algorithm
    self.AlgTable = {}
    self.rad = WINDOW_WIDTH // ROWS

    # Creates an empty map
    for row in range(ROWS):
      rowList = []
      for col in range(COLUMNS):
        node = Node(row, col, self.rad)
        node.set_free()
        rowList.append(node) # Append white node
      self.map.append(rowList)
    # Display empty map
    WINDOW.fill(WHITE)
    # Add obstacles to map
    self.add_obstacles()
    pygame.display.update()
   
  def set_startNode(self,node):
    # Sets the start node and draws it
    self.start = node
    node.set_start()
    node.draw_node()
    pygame.display.update()

  def set_endNode(self,node):
    # Sets the goal node and draws it
    self.end = node
    node.set_goal()
    node.draw_node()
    pygame.display.update()

  def add_obstacles(self):
    # Generates the number of obstacles decided by the user and draws them on the map

    cur_obstacles_number = 0 # Tracks number of obstacles generated
    while cur_obstacles_number < self.obstacles_number:
      # Initialize top left vertex, rectangle dimensions randomly
      row_ver, col_ver = random.randint(0,self.rows), random.randint(0,self.cols)
      WIDTH, HEIGHT = random.randint(0,int(self.cols/5)), random.randint(0,int(self.rows/5)) # Limit the dimension of the obstacles
      # Perform a check on width and height to avoid getting out of map boundaries
      WIDTH = min(WIDTH, self.cols-col_ver)
      HEIGHT = min(HEIGHT, self.rows-row_ver)
      # Set all nodes in the rectangle region to obstacles
      for row in range(row_ver, row_ver+HEIGHT):
        for col in range(col_ver, col_ver+WIDTH):
          self.map[row][col].set_obstacle()
      # Draw the obstacle on the map
      pygame.draw.rect(WINDOW, BLACK, (col_ver*self.rad,row_ver*self.rad,WIDTH*self.rad,HEIGHT*self.rad))
      pygame.display.update()
      # Update current number of obstacles
      cur_obstacles_number += 1

  def find_nearest(self,rand_node):
    # Given a randomly generated node (as a tuple with its coordinates (x,y)), finds its closest node and corresponding distance
    min_dis = (self.rows**2+self.cols**2)**0.5 # Initialized at maximum possible given grid dimension
    for node in self.AlgTable.keys():
      dis = norm2(rand_node, node)
      if dis < min_dis:
        min_dis = dis
        nearest_node = node
    return nearest_node, min_dis  

  def obstacles_on_the_way(self, rand_node, nearest_node, dis):
    # Checks if there are obstacles between two given nodes coordinates
    row_rand, col_rand = rand_node
    row_node, col_node = nearest_node

    # The distance between the new nodes is discretized in 20 points (trial and error) and each sampled point is checked to see if it is an obstacle
    for i in range(1,20):
      step = i/20
      col_new = round(col_node*step + col_rand*(1-step))
      row_new = round(row_node*step + row_rand*(1-step))
      if self.map[row_new][col_new].is_obstacle():
        return True

  def add_new_node(self, rand_node, nearest_node, dis):
    # Given the randomly generated node and its closest node and its distance, the new node to be added position is computed
    row_rand, col_rand = rand_node
    row_node, col_node = nearest_node

    # If the nodes are closer than the step size, directly use the randomly generated node
    if dis < self.step_size:
      row_new, col_new = rand_node
    # Otherwise, place the node at a distance step size from nearest node along the direction connecting the randomly generated node and its closest
    else:
      th = math.atan2(row_rand - row_node, col_rand - col_node)
      col_new = round(col_node + self.step_size*math.cos(th))
      row_new = round(row_node + self.step_size*math.sin(th))

    if self.map[row_new][col_new].is_obstacle(): # In case during the check (due to roundings) this exact spot went unnoticed
      return
  
    # If the new node is close enough to the goal node, directly use the goal node
    if norm2((row_new, col_new), self.end.get_pos()) < self.step_size:
      row_new, col_new = self.end.get_pos()
      self.goal_found = True

    # Add the newly found node to the table, draw it and connect it to its closest node
    self.AlgTable[(row_new,col_new)] = nearest_node
    self.map[row_new][col_new].set_node()
    self.map[row_new][col_new].draw_node()
    pygame.draw.line(WINDOW, RED, (self.map[row_new][col_new].x ,self.map[row_new][col_new].y) , (self.map[row_node][col_node].x ,self.map[row_node][col_node].y))
    pygame.display.update()

  def retrieve_path(self):
    current = self.end.get_pos()
    while current != self.start.get_pos():
      self.map[current[0]][current[1]].set_path()
      self.map[current[0]][current[1]].draw_node()
      pygame.draw.line(WINDOW, PURPLE, (self.map[current[0]][current[1]].x, self.map[current[0]][current[1]].y), (self.map[self.AlgTable[current][0]][self.AlgTable[current][1]].x, self.map[self.AlgTable[current][0]][self.AlgTable[current][1]].y))
      current = self.AlgTable[current]
      pygame.display.update()

  def RRT_path_finder(self):
    # Implements RRT algorithm
    # Only active node at the start is the start node
    self.AlgTable[self.start.get_pos()] = None

    while not self.goal_found:
      # Initialize a random node
      row_rand, col_rand = random.randint(0,self.rows-1), random.randint(0,self.cols-1)
      # Verify it is not an obstacle
      if self.map[row_rand][col_rand].is_obstacle() or (row_rand, col_rand) in self.AlgTable.keys():
        continue
      # Find its nearest node
      nearest_node, distance = self.find_nearest((row_rand,col_rand))
      # Verify there are no obstacles on the way
      if self.obstacles_on_the_way((row_rand,col_rand), nearest_node, distance):
        continue
      # Add new node to the tree
      self.add_new_node((row_rand,col_rand), nearest_node, distance)
      # When goal is found, retrieve entire path
      if self.goal_found:
        self.retrieve_path()

In [7]:
def main():
    DIM = 50 # Maze dimension (square)
    step_size = 3 # Max distance from closest node at which new node can be place
    N_OBSTACLES = 30 # Number of obstacles in the map
    # Create a maze instance
    myMaze = Maze(DIM, DIM, N_OBSTACLES, step_size)

    started = False
    run = True

    while run:
        for event in pygame.event.get():  # Detects actions while display ON
            if event.type == pygame.QUIT: 
                run = False

            if pygame.mouse.get_pressed()[0]: # If left mouse button is pressed
                pos = pygame.mouse.get_pos()
                row,col = get_clicked_pos(pos, DIM) # Retrieve row, col in maze
                spot = myMaze.map[row][col] # Get the node in the clicked position
                # In case the start was not defined, make it
                if not myMaze.start and not spot.is_goal() and not spot.is_obstacle():
                    myMaze.set_startNode(spot)
                # In case the end was not defined, make it
                elif not myMaze.end and not spot.is_start() and not spot.is_obstacle():
                    myMaze.set_endNode(spot)

            elif pygame.mouse.get_pressed()[2]: # If right mouse button is pressed
                pos = pygame.mouse.get_pos()
                row,col = get_clicked_pos(pos, DIM) # Retrieve row, col in maze
                spot = myMaze.map[row][col] # Get the node in the clicked position
                spot_pos = spot.get_pos()
                # Reset the spot unless it is an obstacle
                if myMaze.start:
                    if norm2(spot.get_pos(), myMaze.start.get_pos()) < myMaze.start.radius:
                        myMaze.start.set_free()
                        print(myMaze.start.color)
                        print(myMaze.start.row, myMaze.start.col)
                        myMaze.map[myMaze.start.row][myMaze.start.col].draw_node()
                        myMaze.start = None
                        pygame.display.update()
                        
                if myMaze.end:
                    if norm2(spot.get_pos(), myMaze.end.get_pos()) < myMaze.end.radius:
                        myMaze.end.set_free()
                        myMaze.map[myMaze.end.row][myMaze.end.col].draw_node()
                        myMaze.end = None
                        pygame.display.update()
            
            if event.type == pygame.KEYDOWN:
                # If start and end are defined and blank space is pressed, start the A* algorithm
                if event.key == pygame.K_SPACE and not started and myMaze.start and myMaze.end:
                    myMaze.RRT_path_finder()

                if event.key == pygame.K_r and myMaze.start and myMaze.end: # If R is pressed on keyboard, reset everything
                    myMaze = Maze(DIM, DIM, N_OBSTACLES, step_size)

    pygame.quit()    

if __name__ == "__main__":
    main()